In [1]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk, peak_calling
import pyranges as pr
import requests
import os
import pandas as pd
import glob
import pickle

In [2]:
%load_ext lab_black

In [3]:
# get chromosome sizes (hg38)
if not os.path.exists("chromsizes.txt"):
    target_url = (
        "http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes"
    )
    chromsizes = pd.read_csv(target_url, sep="\t", header=None)
    chromsizes.columns = ["Chromosome", "End"]
    chromsizes["Start"] = [0] * chromsizes.shape[0]
    chromsizes = chromsizes.loc[:, ["Chromosome", "Start", "End"]]
    chromsizes = pr.PyRanges(chromsizes)
    chromsizes.to_csv("chromsizes.txt")
    chromsizes
else:
    chromsizes = pd.read_csv("chromsizes.txt")

In [6]:
cto_consensus_paths = sorted(glob.glob(f"male_female_celltypefaircto/*dimreduc.pkl"))
cto_consensus_path_dict = {
    x.split("/")[-1].split(f"_")[0]: x for x in cto_consensus_paths
}
cto_consensus_path_dict

{'10xmultiome': 'male_female_celltypefaircto/10xmultiome_malefemalecelltypefair.model_29topics.dimreduc.pkl',
 '10xv11': 'male_female_celltypefaircto/10xv11_malefemalecelltypefair.model_29topics.dimreduc.pkl',
 '10xv1': 'male_female_celltypefaircto/10xv1_malefemalecelltypefair.model_25topics.dimreduc.pkl',
 '10xv2': 'male_female_celltypefaircto/10xv2_malefemalecelltypefair.model_30topics.dimreduc.pkl',
 'ddseq': 'male_female_celltypefaircto/ddseq_malefemalecelltypefair.model_24topics.dimreduc.pkl',
 'hydrop': 'male_female_celltypefaircto/hydrop_malefemalecelltypefair.model_29topics.dimreduc.pkl',
 'mtscatac': 'male_female_celltypefaircto/mtscatac_malefemalecelltypefair.model_26topics.dimreduc.pkl',
 's3atac': 'male_female_celltypefaircto/s3atac_malefemalecelltypefair.model_24topics.dimreduc.pkl'}

In [7]:
fragments_path_dict = {
    x: "/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz"
    for x in cto_consensus_path_dict.keys()
}
fragments_path_dict

{'10xmultiome': '/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz',
 '10xv11': '/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz',
 '10xv1': '/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz',
 '10xv2': '/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz',
 'ddseq': '/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz',
 'hydrop': '/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz',
 'mtscatac': '/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz',
 's3atac': '/lustre1/project/stg_00090/scatac_benchmark/fixedcells_4_merged/merged_all_1.fragments.ID.sorted.tsv.gz'}

In [9]:
fragments_path_dict = {
    x.split("/")[-1].split(f".fragments.tsv.gz")[0]: x
    for x in sorted(glob.glob("../1_data_repository/fixedcells_fragments/*.tsv.gz"))
}
fragments_path_dict

{'BIO_ddseq_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_1.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_2.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_3.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_3.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_4.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_4.FIXEDCELLS.fragments.tsv.gz',
 'BRO_mtscatac_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BRO_mtscatac_1.FIXEDCELLS.fragments.tsv.gz',
 'BRO_mtscatac_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BRO_mtscatac_2.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xv11_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragmen

In [10]:
import gc
import logging
import os
import re
import subprocess
import sys
from typing import Dict, List, Optional, Union

import numpy as np
import pandas as pd
import pyBigWig
import pyranges as pr
import ray

from pycisTopic.cistopic_class import *
from pycisTopic.utils import *


def export_pseudobulk(
    input_data: Union["CistopicObject", pd.DataFrame, Dict[str, pd.DataFrame]],
    variable: str,
    chromsizes: Union[pd.DataFrame, pr.PyRanges],
    bed_path: str,
    bigwig_path: str,
    path_to_fragments: Optional[Dict[str, str]] = None,
    sample_id_col: Optional[str] = "sample_id",
    n_cpu: Optional[int] = 1,
    normalize_bigwig: Optional[bool] = True,
    remove_duplicates: Optional[bool] = True,
    split_pattern: Optional[str] = "___",
    use_polars: Optional[bool] = True,
    **kwargs
):
    """
    Create pseudobulks as bed and bigwig from single cell fragments file given a barcode annotation.
    Parameters
    ---------
    input_data: CistopicObject or pd.DataFrame
            A :class:`CistopicObject` containing the specified `variable` as a column in :class:`CistopicObject.cell_data` or a cell metadata
            :class:`pd.DataFrame` containing barcode as rows, containing the specified `variable` as a column (additional columns are
            possible) and a `sample_id` column. Index names must contain the BARCODE (e.g. ATGTCGTC-1), additional tags are possible separating with -
            (e.g. ATGCTGTGCG-1-Sample_1). The levels in the sample_id column must agree with the keys in the path_to_fragments dictionary.
            Alternatively, if the cell metadata contains a column named barcode it will be used instead of the index names.
    variable: str
            A character string indicating the column that will be used to create the different group pseudobulk. It must be included in
            the cell metadata provided as input_data.
    chromsizes: pd.DataFrame or pr.PyRanges
            A data frame or :class:`pr.PyRanges` containing size of each chromosome, containing 'Chromosome', 'Start' and 'End' columns.
    bed_path: str
            Path to folder where the fragments bed files per group will be saved. If None, files will not be generated.
    bigwig_path: str
            Path to folder where the bigwig files per group will be saved. If None, files will not be generated.
    path_to_fragments: str or dict, optional
            A dictionary of character strings, with sample name as names indicating the path to the fragments file/s from which pseudobulk profiles have to
            be created. If a :class:`CistopicObject` is provided as input it will be ignored, but if a cell metadata :class:`pd.DataFrame` is provided it
            is necessary to provide it. The keys of the dictionary need to match with the sample_id tag added to the index names of the input data frame.
    sample_id_col: str, optional
            Name of the column containing the sample name per barcode in the input :class:`CistopicObject.cell_data` or class:`pd.DataFrame`. Default: 'sample_id'.
    n_cpu: int, optional
            Number of cores to use. Default: 1.
    normalize_bigwig: bool, optional
            Whether bigwig files should be CPM normalized. Default: True.
    remove_duplicates: bool, optional
            Whether duplicates should be removed before converting the data to bigwig.
    split_pattern: str, optional
            Pattern to split cell barcode from sample id. Default: ___ .
    use_polars: bool, optional
            Whether to use polars to read fragments files. Default: True.
    **kwargs
            Additional parameters for ray.init()
    Return
    ------
    dict
            A dictionary containing the paths to the newly created bed fragments files per group a dictionary containing the paths to the
            newly created bigwig files per group.
    """
    # Create logger
    level = logging.INFO
    log_format = "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
    handlers = [logging.StreamHandler(stream=sys.stdout)]
    logging.basicConfig(level=level, format=log_format, handlers=handlers)
    log = logging.getLogger("cisTopic")

    # Get fragments file
    if isinstance(input_data, CistopicObject):
        path_to_fragments = input_data.path_to_fragments
        if path_to_fragments is None:
            log.error("No path_to_fragments in this cisTopic object.")
        cell_data = input_data.cell_data
    elif isinstance(input_data, pd.DataFrame):
        if path_to_fragments is None:
            log.error("Please, provide path_to_fragments.")
        cell_data = input_data
    # Check for sample_id column
    try:
        sample_ids = list(set(cell_data[sample_id_col]))
    except ValueError:
        print(
            'Please, include a sample identification column (e.g. "sample_id") in your cell metadata!'
        )

    # Get fragments
    fragments_df_dict = {}
    for sample_id in path_to_fragments.keys():
        if sample_id not in sample_ids:
            log.info(
                "The following path_to_fragments entry is not found in the cell metadata sample_id_col: ",
                sample_id,
                ". It will be ignored.",
            )
        else:
            log.info("Reading fragments from " + path_to_fragments[sample_id])
            fragments_df = read_fragments_from_file(
                path_to_fragments[sample_id], use_polars=use_polars
            ).df
            # Convert to int32 for memory efficiency
            fragments_df.Start = np.int32(fragments_df.Start)
            fragments_df.End = np.int32(fragments_df.End)
            if "Score" in fragments_df:
                fragments_df.Score = np.int32(fragments_df.Score)
            if "barcode" in cell_data:
                fragments_df = fragments_df.loc[
                    fragments_df["Name"].isin(cell_data["barcode"].tolist())
                ]
            else:
                fragments_df = fragments_df.loc[
                    fragments_df["Name"].isin(
                        prepare_tag_cells(cell_data.index.tolist(), split_pattern)
                    )
                ]
            fragments_df_dict[sample_id] = fragments_df

    # Set groups
    if "barcode" in cell_data:
        cell_data = cell_data.loc[:, [variable, sample_id_col, "barcode"]]
    else:
        cell_data = cell_data.loc[:, [variable, sample_id_col]]
    cell_data[variable] = cell_data[variable].replace(" ", "", regex=True)
    cell_data[variable] = cell_data[variable].replace("[^A-Za-z0-9]+", "_", regex=True)
    groups = sorted(list(set(cell_data[variable])))
    # Check chromosome sizes
    if isinstance(chromsizes, pd.DataFrame):
        chromsizes = chromsizes.loc[:, ["Chromosome", "Start", "End"]]
        chromsizes = pr.PyRanges(chromsizes)
    # Check that output dir exist and generate output paths
    if isinstance(bed_path, str):
        if not os.path.exists(bed_path):
            os.makedirs(bed_path)
        bed_paths = {
            group: os.path.join(bed_path, str(group) + ".bed.gz") for group in groups
        }
    else:
        bed_paths = {}
    if isinstance(bigwig_path, str):
        if not os.path.exists(bigwig_path):
            os.makedirs(bigwig_path)
        bw_paths = {
            group: os.path.join(bigwig_path, str(group) + ".bw") for group in groups
        }
    else:
        bw_paths = {}
    # Create pseudobulks
    if n_cpu > 1:
        ray.init(num_cpus=n_cpu, **kwargs)
        ray_handle = ray.wait(
            [
                export_pseudobulk_ray.remote(
                    cell_data,
                    group,
                    fragments_df_dict,
                    chromsizes,
                    bigwig_path,
                    bed_path,
                    sample_id_col,
                    normalize_bigwig,
                    remove_duplicates,
                    split_pattern,
                )
                for group in groups
            ],
            num_returns=len(groups),
        )
        ray.shutdown()
    else:
        [
            export_pseudobulk_one_sample(
                cell_data,
                group,
                fragments_df_dict,
                chromsizes,
                bigwig_path,
                bed_path,
                sample_id_col,
                normalize_bigwig,
                remove_duplicates,
                split_pattern,
            )
            for group in groups
        ]

    return bw_paths, bed_paths


def export_pseudobulk_one_sample(
    cell_data: pd.DataFrame,
    group: str,
    fragments_df_dict: Dict[str, pd.DataFrame],
    chromsizes: pr.PyRanges,
    bigwig_path: str,
    bed_path: str,
    sample_id_col: Optional[str] = "sample_id",
    normalize_bigwig: Optional[bool] = True,
    remove_duplicates: Optional[bool] = True,
    split_pattern: Optional[str] = "___",
):
    """
    Create pseudobulk as bed and bigwig from single cell fragments file given a barcode annotation and a group.
    Parameters
    ---------
    cell_data: pd.DataFrame
            A cell metadata :class:`pd.Dataframe` containing barcodes, their annotation and their sample of origin.
    group: str
            A character string indicating the group for which pseudobulks will be created.
    fragments_df_dict: dict
            A dictionary containing data frames as values with 'Chromosome', 'Start', 'End', 'Name', and 'Score' as columns; and sample label
            as keys. 'Score' indicates the number of times that a fragments is found assigned to that barcode.
    chromsizes: pr.PyRanges
            A :class:`pr.PyRanges` containing size of each column, containing 'Chromosome', 'Start' and 'End' columns.
    bigwig_path: str
            Path to folder where the bigwig file will be saved.
    bed_path: str
            Path to folder where the fragments bed file will be saved.
    sample_id_col: str, optional
            Name of the column containing the sample name per barcode in the input :class:`CistopicObject.cell_data` or class:`pd.DataFrame`. Default: 'sample_id'.
    normalize_bigwig: bool, optional
            Whether bigwig files should be CPM normalized. Default: True.
    remove_duplicates: bool, optional
            Whether duplicates should be removed before converting the data to bigwig.
    split_pattern: str
            Pattern to split cell barcode from sample id. Default: ___ .
    """
    # Create logger
    level = logging.INFO
    log_format = "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
    handlers = [logging.StreamHandler(stream=sys.stdout)]
    logging.basicConfig(level=level, format=log_format, handlers=handlers)
    log = logging.getLogger("cisTopic")

    log.info("Creating pseudobulk for " + str(group))
    group_fragments_list = []
    group_fragments_dict = {}
    for sample_id in fragments_df_dict:
        sample_data = cell_data[cell_data.loc[:, sample_id_col].isin([sample_id])]
        if "barcode" in sample_data:
            sample_data.index = sample_data["barcode"].tolist()
        else:
            sample_data.index = prepare_tag_cells(
                sample_data.index.tolist(), split_pattern
            )
        group_var = sample_data.iloc[:, 0]
        barcodes = group_var[group_var.isin([group])].index.tolist()
        fragments_df = fragments_df_dict[sample_id]
        group_fragments = fragments_df.loc[fragments_df["Name"].isin(barcodes)]
        if len(fragments_df_dict) > 1:
            group_fragments_dict[sample_id] = group_fragments

    if len(fragments_df_dict) > 1:
        group_fragments_list = [
            group_fragments_dict[list(group_fragments_dict.keys())[x]]
            for x in range(len(fragments_df_dict))
        ]
        group_fragments = group_fragments_list[0].append(group_fragments_list[1:])

    group_fragments = group_fragments[
        group_fragments["Chromosome"].isin(chromsizes.Chromosome)
    ]

    del group_fragments_dict
    del group_fragments_list
    del fragments_df
    gc.collect()

    group_pr = pr.PyRanges(group_fragments)
    if isinstance(bigwig_path, str):
        bigwig_path_group = os.path.join(bigwig_path, str(group) + ".bw")
        if remove_duplicates:
            group_pr.to_bigwig(
                path=bigwig_path_group,
                chromosome_sizes=chromsizes,
                rpm=normalize_bigwig,
            )
        else:
            group_pr.to_bigwig(
                path=bigwig_path_group,
                chromosome_sizes=chromsizes,
                rpm=normalize_bigwig,
                value_col="Score",
            )
    if isinstance(bed_path, str):
        bed_path_group = os.path.join(bed_path, str(group) + ".bed.gz")
        group_pr.to_bed(
            path=bed_path_group, keep=False, compression="infer", chain=False
        )

    log.info(str(group) + " done!")


@ray.remote
def export_pseudobulk_ray(
    cell_data: pd.DataFrame,
    group: str,
    fragments_df_dict: Dict[str, pd.DataFrame],
    chromsizes: pr.PyRanges,
    bigwig_path: str,
    bed_path: str,
    sample_id_col: Optional[str] = "sample_id",
    normalize_bigwig: Optional[bool] = True,
    remove_duplicates: Optional[bool] = True,
    split_pattern: Optional[str] = "___",
):
    """
    Create pseudobulk as bed and bigwig from single cell fragments file given a barcode annotation and a group.
    Parameters
    ---------
    cell_data: pd.DataFrame
            A cell metadata :class:`pd.Dataframe` containing barcodes, their annotation and their sample of origin.
    group: str
            A character string indicating the group for which pseudobulks will be created.
    fragments_df_dict: dict
            A dictionary containing data frames as values with 'Chromosome', 'Start', 'End', 'Name', and 'Score' as columns; and sample label
            as keys. 'Score' indicates the number of times that a fragments is found assigned to that barcode.
    chromsizes: pr.PyRanges
            A :class:`pr.PyRanges` containing size of each column, containing 'Chromosome', 'Start' and 'End' columns.
    bed_path: str
            Path to folder where the fragments bed file will be saved.
    bigwig_path: str
            Path to folder where the bigwig file will be saved.
    sample_id_col: str, optional
            Name of the column containing the sample name per barcode in the input :class:`CistopicObject.cell_data` or class:`pd.DataFrame`. Default: 'sample_id'.
    normalize_bigwig: bool, optional
            Whether bigwig files should be CPM normalized. Default: True.
    remove_duplicates: bool, optional
            Whether duplicates should be removed before converting the data to bigwig.
    split_pattern: str
            Pattern to split cell barcode from sample id. Default: ___ .
    """
    export_pseudobulk_one_sample(
        cell_data,
        group,
        fragments_df_dict,
        chromsizes,
        bigwig_path,
        bed_path,
        sample_id_col,
        normalize_bigwig,
        remove_duplicates,
        split_pattern,
    )

In [12]:
# for sample in ["BRO_mtscatac_1.LIBDS"]:
bw_paths_dict = {}
bed_paths_dict = {}

import ray

if ray.is_initialized():
    print("Shutting down Ray")
    ray.shutdown()

for sample in cto_consensus_path_dict.keys():
    supersample = ".".join(sample.split(".")[:2])

    bed_path = os.path.join(
        "final_consensus_peaks", f"{sample}__SCREEN_pseudobulk_bed_files"
    )
    bw_path = os.path.join(
        "final_consensus_peaks", f"{sample}__SCREEN_pseudobulk_bw_files"
    )

    if not os.path.exists(bed_path):
        cto_path = cto_consensus_path_dict[sample]
        with open(cto_path, "rb") as f:
            cto = pickle.load(f)

        fragments_path_dict_sub = {}
        for sample in cto.cell_data["sample_id"].unique():
            fragments_path_dict_sub[sample] = fragments_path_dict[sample]

        print(fragments_path_dict_sub)

        bw_paths, bed_paths = export_pseudobulk(
            input_data=cto,
            variable="harmony_consensus_cell_type__mega",
            sample_id_col="sample_id",
            chromsizes=chromsizes,
            bed_path=bed_path,
            bigwig_path=bw_path,
            path_to_fragments=fragments_path_dict_sub,
            n_cpu=16,
            normalize_bigwig=True,
            remove_duplicates=True,
        )

        if ray.is_initialized():
            print("Shutting down Ray")
            ray.shutdown()
    else:
        print(f"{bed_path} exists, skipping...")

{'SAN_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/SAN_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz', 'VIB_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/VIB_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz', 'VIB_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/VIB_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz', 'CNA_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz', 'SAN_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/SAN_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz', 'CNA_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz'}
2023-03-08 11:56:06,851 cisTopic     INFO     Reading fragments from ../1_data_repository/fixedcells_fragments/SAN_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz


--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/venv/bin/ipython", line 8, in <module>
    sys.exit(start_ipython())
  File "/opt/venv/lib/python3.8/site-packages/IPython/__init__.py", line 124, in start_ipython
    return launch_new_instance(argv=argv, **kwargs)
  File "/opt/venv/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/opt/venv/lib/python3.8/site-packages/IPython/terminal/ipapp.py", line 314,

2023-03-08 11:57:57,589 cisTopic     INFO     Reading fragments from ../1_data_repository/fixedcells_fragments/VIB_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz


--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/venv/bin/ipython", line 8, in <module>
    sys.exit(start_ipython())
  File "/opt/venv/lib/python3.8/site-packages/IPython/__init__.py", line 124, in start_ipython
    return launch_new_instance(argv=argv, **kwargs)
  File "/opt/venv/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/opt/venv/lib/python3.8/site-packages/IPython/terminal/ipapp.py", line 314,

2023-03-08 11:58:20,190 cisTopic     INFO     Reading fragments from ../1_data_repository/fixedcells_fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz


...

(export_pseudobulk_ray pid=1189605) 2023-03-08 13:18:52,331 cisTopic     INFO     Creating pseudobulk for Naturalkillercell


(export_pseudobulk_ray pid=1189594) /tmp/ipykernel_1165301/2558041373.py:272: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(export_pseudobulk_ray pid=1189603) /tmp/ipykernel_1165301/2558041373.py:272: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(export_pseudobulk_ray pid=1189605) /tmp/ipykernel_1165301/2558041373.py:272: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


(export_pseudobulk_ray pid=1189597) 2023-03-08 13:19:09,862 cisTopic     INFO     CD16_monocyte done!
(export_pseudobulk_ray pid=1189603) 2023-03-08 13:19:27,528 cisTopic     INFO     Dendriticcell done!
(export_pseudobulk_ray pid=1189605) 2023-03-08 13:20:08,597 cisTopic     INFO     Naturalkillercell done!
(export_pseudobulk_ray pid=1189604) 2023-03-08 13:22:02,799 cisTopic     INFO     Bcell done!
(export_pseudobulk_ray pid=1189594) 2023-03-08 13:23:41,602 cisTopic     INFO     CytotoxicTcell done!
(export_pseudobulk_ray pid=1189608) 2023-03-08 13:27:40,260 cisTopic     INFO     CD4_Tcell done!
{'OHS_s3atac_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/OHS_s3atac_1.FIXEDCELLS.fragments.tsv.gz'}
2023-03-08 13:28:40,879 cisTopic     INFO     Reading fragments from ../1_data_repository/fixedcells_fragments/OHS_s3atac_1.FIXEDCELLS.fragments.tsv.gz


--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/venv/bin/ipython", line 8, in <module>
    sys.exit(start_ipython())
  File "/opt/venv/lib/python3.8/site-packages/IPython/__init__.py", line 124, in start_ipython
    return launch_new_instance(argv=argv, **kwargs)
  File "/opt/venv/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/opt/venv/lib/python3.8/site-packages/IPython/terminal/ipapp.py", line 314,

(export_pseudobulk_ray pid=1192458) 2023-03-08 13:29:54,846 cisTopic     INFO     Creating pseudobulk for Bcell
(export_pseudobulk_ray pid=1192459) 2023-03-08 13:29:55,105 cisTopic     INFO     Creating pseudobulk for CD14_monocyte
(export_pseudobulk_ray pid=1192470) 2023-03-08 13:29:55,294 cisTopic     INFO     Creating pseudobulk for CD16_monocyte
(export_pseudobulk_ray pid=1192462) 2023-03-08 13:29:55,543 cisTopic     INFO     Creating pseudobulk for CD4_Tcell
(export_pseudobulk_ray pid=1192460) 2023-03-08 13:29:55,703 cisTopic     INFO     Creating pseudobulk for CytotoxicTcell
(export_pseudobulk_ray pid=1192464) 2023-03-08 13:29:55,940 cisTopic     INFO     Creating pseudobulk for Dendriticcell
(export_pseudobulk_ray pid=1192469) 2023-03-08 13:29:56,251 cisTopic     INFO     Creating pseudobulk for Naturalkillercell
(export_pseudobulk_ray pid=1192464) 2023-03-08 13:30:07,562 cisTopic     INFO     Dendriticcell done!
(export_pseudobulk_ray pid=1192470) 2023-03-08 13:30:09,004 cisTo

In [19]:
cto.cell_data["fmx_sample_harmony_consensus_cell_type__mega"] = (
    cto.cell_data["fmx_sample"]
    + "_"
    + [x.replace(" ", "_") for x in cto.cell_data["harmony_consensus_cell_type__mega"]]
)

In [23]:
# for sample in ["BRO_mtscatac_1.LIBDS"]:
bw_paths_dict = {}
bed_paths_dict = {}

import ray

if ray.is_initialized():
    print("Shutting down Ray")
    ray.shutdown()

for sample in cto_consensus_path_dict.keys():
    supersample = ".".join(sample.split(".")[:2])

    bed_path = os.path.join(
        "male_female_coverage", f"{sample}__SCREEN_pseudobulk_bed_files_nonnorm"
    )
    bw_path = os.path.join(
        "male_female_coverage", f"{sample}__SCREEN_pseudobulk_bw_files_nonnorm"
    )

    if not os.path.exists(bed_path):
        cto_path = cto_consensus_path_dict[sample]
        with open(cto_path, "rb") as f:
            cto = pickle.load(f)

        fragments_path_dict_sub = {}
        for sample in cto.cell_data["sample_id"].unique():
            fragments_path_dict_sub[sample] = fragments_path_dict[sample]

        print(fragments_path_dict_sub)

        cto.cell_data["fmx_sample_harmony_consensus_cell_type__mega"] = (
            cto.cell_data["fmx_sample"]
            + "_"
            + [
                x.replace(" ", "_")
                for x in cto.cell_data["harmony_consensus_cell_type__mega"]
            ]
        )

        bw_paths, bed_paths = export_pseudobulk(
            input_data=cto,
            variable="fmx_sample_harmony_consensus_cell_type__mega",
            sample_id_col="sample_id",
            chromsizes=chromsizes,
            bed_path=bed_path,
            bigwig_path=bw_path,
            path_to_fragments=fragments_path_dict_sub,
            n_cpu=16,
            normalize_bigwig=False,
            remove_duplicates=True,
        )

        if ray.is_initialized():
            print("Shutting down Ray")
            ray.shutdown()
    else:
        print(f"{bed_path} exists, skipping...")

{'SAN_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/SAN_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz', 'VIB_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/VIB_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz', 'VIB_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/VIB_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz', 'CNA_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz', 'SAN_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/SAN_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz', 'CNA_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz'}
2023-03-08 14:24:09,987 cisTopic     INFO     Reading fragments from ../1_data_repository/fixedcells_fragments/SAN_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz


--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/venv/bin/ipython", line 8, in <module>
    sys.exit(start_ipython())
  File "/opt/venv/lib/python3.8/site-packages/IPython/__init__.py", line 124, in start_ipython
    return launch_new_instance(argv=argv, **kwargs)
  File "/opt/venv/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/opt/venv/lib/python3.8/site-packages/IPython/terminal/ipapp.py", line 314,

2023-03-08 14:25:52,469 cisTopic     INFO     Reading fragments from ../1_data_repository/fixedcells_fragments/VIB_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz


--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/venv/bin/ipython", line 8, in <module>
    sys.exit(start_ipython())
  File "/opt/venv/lib/python3.8/site-packages/IPython/__init__.py", line 124, in start_ipython
    return launch_new_instance(argv=argv, **kwargs)
  File "/opt/venv/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/opt/venv/lib/python3.8/site-packages/IPython/terminal/ipapp.py", line 314,

2023-03-08 14:26:13,864 cisTopic     INFO     Reading fragments from ../1_data_repository/fixedcells_fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz


...

(export_pseudobulk_ray pid=1218921) 2023-03-08 15:26:05,229 cisTopic     INFO     Creating pseudobulk for sampleB_Dendritic_cell
(export_pseudobulk_ray pid=1218907) 2023-03-08 15:26:06,772 cisTopic     INFO     Creating pseudobulk for sampleB_Natural_killer_cell


(export_pseudobulk_ray pid=1218917) /tmp/ipykernel_1165301/2558041373.py:272: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(export_pseudobulk_ray pid=1218921) /tmp/ipykernel_1165301/2558041373.py:272: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(export_pseudobulk_ray pid=1218907) /tmp/ipykernel_1165301/2558041373.py:272: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


(export_pseudobulk_ray pid=1218920) 2023-03-08 15:26:11,692 cisTopic     INFO     sampleA_Dendritic_cell done!
(export_pseudobulk_ray pid=1218919) 2023-03-08 15:26:14,140 cisTopic     INFO     sampleB_CD16_monocyte done!
(export_pseudobulk_ray pid=1218921) 2023-03-08 15:26:25,869 cisTopic     INFO     sampleB_Dendritic_cell done!
(export_pseudobulk_ray pid=1218908) 2023-03-08 15:26:33,486 cisTopic     INFO     sampleA_Natural_killer_cell done!
(export_pseudobulk_ray pid=1218907) 2023-03-08 15:26:44,573 cisTopic     INFO     sampleB_Natural_killer_cell done!
(export_pseudobulk_ray pid=1218916) 2023-03-08 15:27:20,888 cisTopic     INFO     sampleA_B_cell done!
(export_pseudobulk_ray pid=1218910) 2023-03-08 15:27:31,121 cisTopic     INFO     sampleB_B_cell done!
(export_pseudobulk_ray pid=1218911) 2023-03-08 15:27:57,791 cisTopic     INFO     sampleA_Cytotoxic_T_cell done!
(export_pseudobulk_ray pid=1218917) 2023-03-08 15:28:34,730 cisTopic     INFO     sampleB_Cytotoxic_T_cell done!
(exp

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/usr/local/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/usr/local/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/venv/bin/ipython", line 8, in <module>
    sys.exit(start_ipython())
  File "/opt/venv/lib/python3.8/site-packages/IPython/__init__.py", line 124, in start_ipython
    return launch_new_instance(argv=argv, **kwargs)
  File "/opt/venv/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/opt/venv/lib/python3.8/site-packages/IPython/terminal/ipapp.py", line 314,

(export_pseudobulk_ray pid=1222297) 2023-03-08 15:32:20,528 cisTopic     INFO     Creating pseudobulk for sampleA_B_cell
(export_pseudobulk_ray pid=1222309) 2023-03-08 15:32:20,762 cisTopic     INFO     Creating pseudobulk for sampleA_CD14_monocyte
(export_pseudobulk_ray pid=1222308) 2023-03-08 15:32:21,081 cisTopic     INFO     Creating pseudobulk for sampleA_CD16_monocyte
(export_pseudobulk_ray pid=1222294) 2023-03-08 15:32:21,270 cisTopic     INFO     Creating pseudobulk for sampleA_CD4_T_cell
(export_pseudobulk_ray pid=1222301) 2023-03-08 15:32:21,549 cisTopic     INFO     Creating pseudobulk for sampleA_Cytotoxic_T_cell
(export_pseudobulk_ray pid=1222305) 2023-03-08 15:32:21,778 cisTopic     INFO     Creating pseudobulk for sampleA_Dendritic_cell
(export_pseudobulk_ray pid=1222307) 2023-03-08 15:32:22,021 cisTopic     INFO     Creating pseudobulk for sampleA_Natural_killer_cell
(export_pseudobulk_ray pid=1222295) 2023-03-08 15:32:22,267 cisTopic     INFO     Creating pseudobulk fo